In [31]:
from transformers import AutoImageProcessor, TFViTModel
import cv2
import tensorflow as tf
import keras

In [32]:
class_names = ["angry","happy","sad"]

In [33]:
resize_rescale_layer = keras.models.Sequential(
    [
        keras.layers.Rescaling(scale=1./255)
    ]
)

In [34]:
#image = cv2.imread("C:\\ajay_ml\\Human Emotions Detection\\train\\angry\\82858.jpg") - result angry
image = cv2.imread("C:\\ajay_ml\\Human Emotions Detection\\train\\sad\\4998.jpg") #- result angry
image = cv2.resize(image,(224,224))
print('image.shape :{}'.format(image.shape))
image = resize_rescale_layer(image)
print('image.shape :{}'.format(image.shape))

image.shape :(224, 224, 3)
image.shape :(224, 224, 3)


In [35]:
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
model = TFViTModel.from_pretrained("google/vit-base-patch16-224-in21k")

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
All PyTorch model weights were used when initializing TFViTModel.

All the weights of TFViTModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


In [36]:
inputs = image_processor(image, return_tensors="tf",do_rescale=False)
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state
list(last_hidden_states.shape)
print(last_hidden_states.shape)

(1, 197, 768)


In [37]:
x = last_hidden_states[:,0,:]
x = keras.layers.Dense(units=3,activation="softmax")(x)

predicted_label = int(tf.math.argmax(x, axis=-1))
print(class_names[predicted_label])

sad
